In [1]:
from greet import greet
from destructure import destructure
import json
import math
import bisect

In [2]:
greet('freddy')

'Hi there, freddy'

In [3]:
animals = ['dog', 'cat', 'tiger']
pairs = enumerate(animals)
reverse = {key: index for (index, key) in pairs}
reverse

{'cat': 1, 'dog': 0, 'tiger': 2}

In [4]:
animals[0] = 'canine'
print('animals', animals)

animals ['canine', 'cat', 'tiger']


In [5]:
animals[1:3] = ['feline', 'squid', 'sharks']
animals

['canine', 'feline', 'squid', 'sharks']

In [6]:
nums = ['one','two','three','four','five','six','seven','eight','nine','ten']
odds = nums[::2]
evens = nums[1::2]

print('evens', evens)
print('odds', odds)

backwards = nums[::-1]
print('backwards', backwards)

evens ['two', 'four', 'six', 'eight', 'ten']
odds ['one', 'three', 'five', 'seven', 'nine']
backwards ['ten', 'nine', 'eight', 'seven', 'six', 'five', 'four', 'three', 'two', 'one']


### Both zip and enumerate produce single use iterators.
Inline is always better...

In [7]:
# an enumeration cannot be reused and we used pairs above
forward = {index: key for (index, key) in pairs}
forward  

{}

In [8]:
# better to inline the enumerate
forward = {index: key for (index, key) in enumerate(animals)}
print('forward', forward)
reverse = {key: index for (index, key) in enumerate(animals)}
print('reverse', reverse)

forward {0: 'canine', 1: 'feline', 2: 'squid', 3: 'sharks'}
reverse {'canine': 0, 'feline': 1, 'squid': 2, 'sharks': 3}


### comprehensions are nice

In [9]:
evens_squared = [a**2 for a in range(2,10,2)]
print(evens_squared)

[4, 16, 36, 64]


In [10]:
pets = ['dog','cat', 'parrot']
counts = [2,3,0]
pet_lookup = {pet:count for (pet, count) in zip(pets, counts) if count > 0}
print(pet_lookup)

{'dog': 2, 'cat': 3}


In [11]:
pc = zip(pets, counts)
for l in pc:
    print(l)

('dog', 2)
('cat', 3)
('parrot', 0)


In [12]:
for l in pc:
    print(l)

don't destructure if you don't have to

In [13]:
groups = [('fred', 'ted'), ('blue', 'orange')]
phrases = ['%s and %s' % group for group in groups]
print(phrases)

['fred and ted', 'blue and orange']


### Note
Assignment to a closure scope variable requires use of the nonlocal keyword. Mutating the contents of an object does not.

In [14]:
def create_greeter(greeting = 'Hi'):
    count = 0
    def greet(name):
        nonlocal count
        count += 1
        return '%s %s %d' % (greeting, name, count)
    
    return greet
    
formal_greeter = create_greeter('Hello there,')
print(formal_greeter('Fred'))
print(formal_greeter('Ted'))

informal_greeter = create_greeter('Hey')
print(informal_greeter('Fred'))
print(informal_greeter('Ted'))



Hello there, Fred 1
Hello there, Ted 2
Hey Fred 1
Hey Ted 2


In [15]:
def create_greeter(greeting = 'Hi'):
    context = {'count':0}
    def greet(name):
        context['count'] += 1
        return '%s %s %d' % (greeting, name, context['count'])
    
    return greet
    
formal_greeter = create_greeter('Hello there,')
print(formal_greeter('Fred'))
print(formal_greeter('Ted'))

informal_greeter = create_greeter('Hey')
print(informal_greeter('Fred'))
print(informal_greeter('Ted'))
print(formal_greeter('Red'))
print(informal_greeter('Red'))



Hello there, Fred 1
Hello there, Ted 2
Hey Fred 1
Hey Ted 2
Hello there, Red 3
Hey Red 3


### Passing function arguments 
Behaves like passing a copy of the reference.
Assignments are lost. Changes to the contents of a complex type bubble out of the function. Like all the well behaved languages!

In [16]:
def add_with_side_effects_lost1(a,b):
    temp_a = a
    a = a + 10
    return temp_a + b

first = 4
second = 6
print('sum of 4 and 6', add_with_side_effects_lost1(first, second))
print('first', first)

sum of 4 and 6 10
first 4


In [17]:
def add_with_side_effects(numbers):
    temp_first = numbers[0]
    numbers[0] += 10
    return temp_first + numbers[1]

nmbrs = [4,6]
print('sum of 4 and 6', add_with_side_effects(nmbrs))
print('nmbrs', nmbrs)

sum of 4 and 6 10
nmbrs [14, 6]


In [18]:
def add_with_side_effects_lost2(numbers):
    temp_sum = numbers[0] + numbers[1]
    numbers = [99, 199]
    return temp_sum

nmbrs = [4,6]
print('sum of 4 and 6', add_with_side_effects_lost2(nmbrs))
print('nmbrs', nmbrs)

sum of 4 and 6 10
nmbrs [4, 6]


### Gather, spread and destructuring work 

In [19]:
def print_ab(a,b):
    print('a',a)
    print('b',b)
    
dogs = ['rex', 'max']
# list of dogs are spread into arguments
print_ab(*dogs)

a rex
b max


In [20]:
# last n arguments will be gathered into a list
def add(name, number, *values):
    return sum(values)

add('fred', 12, 1,2,3)

6

In [21]:
numbers = [1,2,3]
# list of numbers will be spread into arguments then gathered into list
# cause we can
add('fred', 12, *numbers)

6

In [22]:
# destructuring 
[a,b,c] = numbers
print(b)

2


### named and default function parameters work 
But once you start specifying defaults in the list you have to keep specifying them

In [23]:
def printem(a, b='B', c='C'):
    print('a',a, 'b',b, 'c',c)
    
printem(b='Bee', a='A')

a A b Bee c C


### Beware mutable defaults
Single default value is created when module loads 

In [24]:
def load_data(raw, default={}):
    if not raw:
        return default
    
    try:
        return json.loads(raw)
    except ValueError:
        return default
        
data1 = load_data('{"name":"Fred"}')
data2 = load_data('{"name":"Ted"}')
data2['name'] = 'TED'
print('data1', data1)
print('data2', data2)


data1 {'name': 'Fred'}
data2 {'name': 'TED'}


In [25]:
data1 = load_data(None)
data2 = load_data(None)
data2['name'] = 'TED'
print('data1', data1)
print('data2', data2)

data1 {'name': 'TED'}
data2 {'name': 'TED'}


In [26]:
def load_data(raw=None, default={}):
    """Always returns a fresh mutation safe dictionary"""
    if not raw:
        return json.loads(json.dumps(default))
    
    try:
        return json.loads(raw)
    except ValueError:
        return json.loads(json.dumps(default))

In [27]:
my_dictionary = {'name': 'Fred'}
passed_default = load_data(None, my_dictionary)
my_dictionary['height'] = 70
print('passed_default', passed_default)
print('my_dictionary', my_dictionary)

passed_default {'name': 'Fred'}
my_dictionary {'name': 'Fred', 'height': 70}


In [28]:
default1 = load_data('{"crap')
default2 = load_data()
default1['height'] = 70
print('default1', default1, 'default2', default2)

default1 {'height': 70} default2 {}


### Force caller to name all but the first required parameter
aka keyword only arguments

In [29]:
def change(raw, *, uppercase = False, reverse = False):
    results = raw.upper() if uppercase else raw
    results = results[::-1] if reverse else results
    return results
    
print(change('Fred'))
print(change('Fred', uppercase = True))
print(change('Fred', reverse = True))
print(change('Fred', reverse = True, uppercase=True))

Fred
FRED
derF
DERF


### Destructuring isn't as easy for dictionaries
Score 1 for JavaScript...

### Lots of string production
F strings give you readability but no formatting   
printf style creation is flexible   

In [30]:
fred = {'name': 'Fred', 'weight': 188, 'height': 70.23}
wilma = {'name': 'Wilma', 'weight': None, 'height': 64.65}
(name, height) = destructure(fred, 'name', 'height')
a = f'''
{name} is {height} inches tall
{name} likes nifty multiline templating
'''
print(a)


Fred is 70.23 inches tall
Fred likes nifty multiline templating



In [31]:
a = '''
%s is %.1f inches tall
He likes multiline strings
''' % (fred['name'], fred['height'])

print(a)


Fred is 70.2 inches tall
He likes multiline strings



In [32]:
a[0:8] # escape characters count

'\nFred is'

In [33]:
a[1:12:2]  # stride works

'Fe s7.'

In [34]:
with_escapes = 'hi there\nhow are you?'
print(with_escapes)

hi there
how are you?


In [35]:
with_no_escapes = 'hi there\\nhow are you?This is a slash \\.'
print(with_no_escapes)

hi there\nhow are you?This is a slash \.


In [36]:
with_no_escapes = r'hi there\nhow are you?This is a slash \.'
# r tells it to ignore specials. r means raw. cause python is dense.
print(with_no_escapes)

hi there\nhow are you?This is a slash \.


### unicode is as simple as possible

In [37]:
spanish_utf8 = b'espa\xc3\xb1ol'
spanish_string = spanish_utf8.decode('utf-8')
print('spanish_utf8', spanish_utf8, 'spanish_string', spanish_string)

spanish_utf8 b'espa\xc3\xb1ol' spanish_string español


### logical and and or works in all possible ways...
But pythonistas use 'and' and 'or'

In [38]:
True & True and (False | True) & (True or False)

True

In [39]:
True and True and (False or True) and (True or False)

True

### You don't cast. You call a function

In [40]:
int(3.14)

3

In [41]:
bool(1)

True

In [42]:
bool(0)

False

In [43]:
int(True)

1

In [44]:
# so cool. floor division is built in
5 // 2 == math.floor(5/2)

True

### Ranges are flexible and convert nicely to lists 

Unlike iterators, ranges never get tired

In [45]:
evens = range(0, 21, 2)
[a for a in evens]
[a for a in evens]  

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

You can manipulate ranges

In [46]:
[a for a in evens[2:6]]

[4, 6, 8, 10]

### Tuples are read only and very handy

In [47]:
critters = 'dog', 'cat'  # parentheses are optional
critters

('dog', 'cat')

In [48]:
things = ('thimble', 'gimble') # i like them
things

('thimble', 'gimble')

In [49]:
# this makes sense to me
(a, b) = (1,2)
print('a', a, 'b', b)
(b, a) = (a, b)
print('a', a, 'b', b)

a 1 b 2
a 2 b 1


In [50]:
a, b = b, a # works fine but looks goofy to me
print('a', a, 'b', b)

a 1 b 2


### multiplying a list or tuple makes copies of the reference, not a deep copy

In [51]:
numbers = ([1, 2, 3], [4,5,6])
lots = numbers * 3
lots

([1, 2, 3], [4, 5, 6], [1, 2, 3], [4, 5, 6], [1, 2, 3], [4, 5, 6])

In [52]:
lots[1][1] = 55
lots

([1, 2, 3], [4, 55, 6], [1, 2, 3], [4, 55, 6], [1, 2, 3], [4, 55, 6])

In [53]:
[a+b+c for a, b, c in lots]

[6, 65, 6, 65, 6, 65]

In [54]:
# rest works in comprehensions
[[a, rest] for a, *rest in lots]

[[1, [2, 3]],
 [4, [55, 6]],
 [1, [2, 3]],
 [4, [55, 6]],
 [1, [2, 3]],
 [4, [55, 6]]]

In [55]:
[a for a in lots]

[[1, 2, 3], [4, 55, 6], [1, 2, 3], [4, 55, 6], [1, 2, 3], [4, 55, 6]]

In [56]:
[a for a, *_ in lots]  # traditional to ignore rest with underscore

[1, 4, 1, 4, 1, 4]

In [57]:
t = (1,2,2,3)
print('how many 1s',t.count(1))
print('how many 2s',t.count(2))
print('how many', len(t))

how many 1s 1
how many 2s 2
how many 4


In [58]:
3 in t

True

In [59]:
t.index(2)

1

In [60]:
a = [2,3,1,5,2,1,3,6,7,4,5,1]
a.sort()
a

[1, 1, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7]

In [61]:
bisect.insort(a,3.2)
a

[1, 1, 1, 2, 2, 3, 3, 3.2, 4, 5, 5, 6, 7]